In [286]:
import requests
import bs4
from bs4 import BeautifulSoup as soup
import json
import pandas
from pandas import DataFrame
import spacy
from spacy.matcher import Matcher
from spacy.tokens import Span
from spacy import displacy
from collections import defaultdict
nlp = spacy.load('en_core_web_md')
from string import punctuation
import itertools

In [287]:
from collections import defaultdict

In [288]:
def get_ld_json(url: str) -> dict:
    parser = "html.parser"
    req = requests.get(url)
    page = soup(req.text, parser)
    return json.loads("".join(page.find("script", {"type":"application/ld+json"}).contents))
def get_ingredientScript(url):
    jsonld = get_ld_json(url)
    useful = jsonld[1]
    ingredients = useful["recipeIngredient"]
    return ingredients
def parse_ingredients(url):
    ingredients = get_ingredientScript(url)
    df = DataFrame (ingredients,columns=['ingredients'])
    # 2 cases
    # "number" ("()") ("unit") (adjective) "noun/subject - ingredient" (, other)
    # contains "to taste"
    df = df["ingredients"]
    df_taste = df[df.str.contains('to taste', case = False)]
    
    df_unit = df[~df.str.contains('to taste', case = False)]
    
    # array of arrays: each array is amount, unit, ingredient, descriptor, preparation
    ingredients_parsed = []
    
    for i in df:
        curr_arr = ["", "", "", "", ""]
        print(i)
        if 'to taste' not in i:
        
            # before we look at POS, remove everything after the comma and put it in prep
            split_string = i.split(", ", 1)
            root_phrase = split_string[0]
            if len(split_string) > 1:
                other_piece = split_string[1]
            else:
                other_piece = ""

            curr_arr[4] = other_piece

            split_string2 = root_phrase.split("(", 1)
            if len(split_string2) > 1:
                split_string3 = split_string2[1].split(")", 1)
                curr_arr[3] = split_string3[0]
                root_phrase = split_string2[0].strip() + split_string3[1]
            doc = nlp(root_phrase)
            index = 0
            for token in doc:
                found_num = False
                # only get first number if it matches criteria
                #if found_num == False and token.pos_ == "NOUN" and not token.is_alpha or token.pos_ == "NUM":
                if index == 0:
                    curr_arr[0] = token.text
                elif index == 1:
                    if token.pos_ != "ADJ":
                        curr_arr[1] = token.text
                    else:
                        curr_arr[3] = token.text
                elif token.dep_ == "ROOT":
                    curr_arr[2] = token.text
                else: 
                    curr_arr[3] = curr_arr[3] + " " + token.text
                    curr_arr[3] = curr_arr[3].strip()
                index+=1
        else:
            i = i.replace("to taste", "")
            doc = nlp(i)
            curr_arr[0] = "to taste"
            for token in doc:
                if token.dep_ == "ROOT":
                    curr_arr[2] = token.text
                else:
                    curr_arr[3] = curr_arr[3] + " " + token.text
                    curr_arr[3] = curr_arr[3].strip()
        ingredients_parsed.append(curr_arr)
        #print(token.text, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop, token.children, token.head)
        #displacy.render(doc, style="dep") # change to serve when we go to python


    
    return ingredients_parsed

In [300]:
url1 = "https://www.allrecipes.com/recipe/232542"

In [301]:
json_ld = get_ld_json(url1)
print(json_ld)

[{'@context': 'http://schema.org', '@type': 'BreadcrumbList', 'itemListElement': [{'@type': 'ListItem', 'position': 1, 'item': {'@id': 'https://www.allrecipes.com/', 'name': 'Home', 'image': None}}, {'@type': 'ListItem', 'position': 2, 'item': {'@id': 'https://www.allrecipes.com/recipes/', 'name': 'Recipes', 'image': None}}, {'@type': 'ListItem', 'position': 3, 'item': {'@id': 'https://www.allrecipes.com/recipes/156/bread/', 'name': 'Bread', 'image': None}}, {'@type': 'ListItem', 'position': 4, 'item': {'@id': 'https://www.allrecipes.com/recipes/360/bread/quick-bread/', 'name': 'Quick Bread Recipes', 'image': None}}, {'@type': 'ListItem', 'position': 5, 'item': {'@id': 'https://www.allrecipes.com/recipes/347/bread/quick-bread/pumpkin-bread/', 'name': 'Pumpkin Bread Recipes', 'image': None}}]}, {'@context': 'http://schema.org', '@type': 'Recipe', 'mainEntityOfPage': 'https://www.allrecipes.com/recipe/232542/low-fat-vegan-pumpkin-chocolate-chip-muffins/', 'name': 'Low-Fat Vegan Pumpkin C

In [302]:
ing = parse_ingredients(url1)
print(ing)

2 cups whole wheat flour
½ cup packed brown sugar
1 tablespoon baking powder
1 teaspoon ground cinnamon
½ teaspoon baking soda
½ teaspoon salt
½ teaspoon ground nutmeg
1 (15 ounce) can solid-pack pumpkin
½ cup water
½ cup chocolate chips
[['2', 'cups', '', 'whole wheat flour', ''], ['½', 'cup', 'sugar', 'packed brown', ''], ['1', 'tablespoon', 'powder', 'baking', ''], ['1', 'teaspoon', 'cinnamon', 'ground', ''], ['½', 'teaspoon', 'soda', 'baking', ''], ['½', 'teaspoon', 'salt', '', ''], ['½', 'teaspoon', 'nutmeg', 'ground', ''], ['1', 'can', '', '15 ounce solid - pack pumpkin', ''], ['½', 'cup', 'water', '', ''], ['½', 'cup', 'chips', 'chocolate', '']]


In [309]:
one_list = itertools.chain.from_iterable(ing)
fil = list(dict.fromkeys(one_list))
new = ['mixture','ingredients']
name = json_ld[1]['name']
for ing in fil:
    for word in ing.split():
        new.append(word)
for w in name.lower().split():
    new.append(w)
new = list(dict.fromkeys(new))
print(new)

['mixture', 'ingredients', 'c', 'h', 'o', 'l', 'a', 't', 'e', 'low-fat', 'vegan', 'pumpkin', 'chocolate', 'chip', 'muffins']


In [310]:
text = json_ld[1]['recipeInstructions']
if len(text) > 1:
    new_text = ''
    for i,t in enumerate(text):
#         print(text[i]['text'])
        new_text = new_text + text[i]['text']
    text = new_text
else:
    text = text[0]['text']
print(text)
print(name)

Preheat oven to 375 degrees F (190 degrees C). Line 12 muffin cups with paper liners.
Whisk whole wheat flour, brown sugar, baking powder, cinnamon, baking soda, salt, and nutmeg in a large bowl. Stir pumpkin and water into dry ingredients, mixing until just moistened; fold in chocolate chips. Spoon batter into prepared muffin cups, filling them to just below the tops.
Bake in the preheated oven until lightly browned and tops of muffins bounce back when pressed lightly, 25 to 30 minutes. Let muffins cool in pans for 5 minutes until removing to a wire rack to cool completely.

Low-Fat Vegan Pumpkin Chocolate Chip Muffins


In [311]:
doc = nlp('Grease a 10-inch tube pan'.lower())
for token in doc:
    print(token.text,token.pos_, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children], token.lemma_)
displacy.render(doc, style="dep")

grease VERB ROOT grease VERB [pan] grease
a DET det pan NOUN [] a
10-inch NUM nummod tube NOUN [] 10-inch
tube NOUN compound pan NOUN [10-inch] tube
pan NOUN dobj grease VERB [a, tube] pan


In [312]:
for chunk in doc.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_,
            chunk.root.head.text,chunk.root.head.pos_)

a 10-inch tube pan pan dobj grease VERB


### Thing that could be tools
#### 1) pattern: lemma: 'in',   

In [313]:
def find_tool(sentence):
    pattern = [{"LEMMA":{"IN":['in','with']}},{"POS":'DET'},{"POS":'NOUN', 'OP':'+'}]
    # tool_pattern = [{"POS":'NOUN', 'OP':'+'}]
    tools = []
    matcher = Matcher(nlp.vocab)
    matcher.add('find', [pattern])
    doc = nlp(sentence)
    matches = matcher(doc)
    if len(matches) != 0:
        span = doc[matches[-1][1]:matches[-1][2]]
        for token in span:
            if token.pos_ == 'NOUN':
                tools.append(token)
    return tools
find_tool('Heat butter in a pan over medium heat')
def find_tool2(sentence):
    in_list = ['in','into','on']
    too =[]
    not_tools = []
    sent = nlp(sentence.lower())
    for chunk in sent.noun_chunks:
#         and chunk.root.head.pos_ != 'VERB' and not chunk.root.is_sent_start
        if chunk.root.text not in new and chunk.root.head.text in in_list:
            too.append(chunk.root.text)
#     for token in sent:
#         if token.text not in too and token.pos_ == 'NOUN':
#             not_tools.append(token.text)
    return too, not_tools
        
    

In [314]:
tools = []

for sentence in nlp(text).sents:
    print(sentence)
    t,nt = find_tool2(str(sentence))
    for each in t:
        print(each)
#         if each not in big_not_tools:
        tools.append(each)
print(tools)
    
    
    

Preheat oven to 375 degrees F (190 degrees C).
Line 12 muffin cups with paper liners.

Whisk whole wheat flour, brown sugar, baking powder, cinnamon, baking soda, salt, and nutmeg in a large bowl.
bowl
Stir pumpkin and water into dry ingredients, mixing until just moistened; fold in chocolate chips.
Spoon batter into prepared muffin cups, filling them to just below the tops.

cups
Bake in the preheated oven until lightly browned and tops of muffins bounce back when pressed lightly, 25 to 30 minutes.
oven
Let muffins cool in pans for 5 minutes until removing to a wire rack to cool completely.

pans
['bowl', 'cups', 'oven', 'pans']
